In [34]:
import shutil
import os
import glob
import random
import json
from collections import defaultdict

In [2]:
def move_datas(origin_root, target_dir, data_extensions=('.jpg')):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    
    for dirpath, dirnames, filenames in os.walk(origin_root):
        for filename in filenames:
            if filename.lower().endswith(data_extensions):
                src_path = os.path.join(dirpath, filename)
                target_path = os.path.join(target_dir, filename)
                
                shutil.copy(src_path, target_path)
                print("완료")

origin_root = 'D:\\Falldown\\Origin\\Training\\SourceData\\Image\\N'
target_dir = 'D:\\Falldown\\Dataset\\Train\\Image'

move_datas(origin_root, target_dir)

완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완

In [26]:
origin_val = 'D:\\Falldown\\Origin\\Validation\\SourceData\\Image\\N'
target_val = 'D:\\Falldown\\Dataset\\Val\\Image'

move_datas(origin_val, target_val)

완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완료
완

In [23]:
# 소스 폴더와 대상 폴더 경로 설정
source_folder = 'D:\\Falldown\\Origin\\Training\\SourceData\\Image\\Y'
train_folder = 'D:\\Falldown\\Dataset\\Train\\Image'
test_folder = 'D:\\Falldown\\Dataset\\Test\\Image'

# 필요한 폴더들이 없으면 생성
for folder in [train_folder, test_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# file_groups에 전체 경로를 저장
file_groups = defaultdict(list)
for file_path in glob.glob(f"{source_folder}\\**\\*.jpg", recursive=True):
    group_key = os.path.basename(file_path).rsplit('_I', 1)[0]
    file_groups[group_key].append(file_path)

# 파일 이동 함수
def copy_files(groups, destination):
    moved_files = 0
    for group in groups:
        for file_path in file_groups[group]:
            dest_path = os.path.join(destination, os.path.basename(file_path))
            shutil.copy(file_path, dest_path)
            moved_files += 1
    return moved_files


# 그룹 목록을 무작위로 섞기
group_keys = list(file_groups.keys())
random.shuffle(group_keys)

# 남은 그룹을 train과 test로 나누기 (각각 1/3씩)
total_groups = len(group_keys)
train_groups = group_keys[:total_groups//3]
test_groups = group_keys[total_groups//3:2*(total_groups//3)]

print(total_groups)

13584


In [31]:
source_val = 'D:\\Falldown\\Origin\\Validation\\SourceData\\Image\\Y'
val_folder = 'D:\\Falldown\\Dataset\\Val\\Image'

# val_groups에 전체 경로를 저장
val_groups = defaultdict(list)
for file_path in glob.glob(f"{source_val}\\**\\*.jpg", recursive=True):
    group_key = os.path.basename(file_path).rsplit('_I', 1)[0]
    val_groups[group_key].append(file_path)

# 그룹 목록을 무작위로 섞기
val_keys = list(val_groups.keys())
random.shuffle(val_keys)

# 1/3만 가져오기 (같은 환자는 다같이 가져오기)
total_val = len(val_keys)
val = val_keys[:total_val//3]

print(total_val)

1704


In [30]:
# 파일 이동 함수
def copy_files(groups, group_dict, destination):
    moved_files = 0
    for group in groups:
        for file_path in group_dict[group]:
            dest_path = os.path.join(destination, os.path.basename(file_path))
            # 파일이 존재하지 않을 때만 복사
            if not os.path.exists(dest_path):
                shutil.copy(file_path, dest_path)
                moved_files += 1
    return moved_files

In [24]:
# train 폴더로 파일 이동
train_files_copied = copy_files(train_groups, file_groups, train_folder)
print(f"Moved {train_files_copied} files to train folder")

# test 폴더로 파일 이동
test_files_copied = copy_files(test_groups, file_groups, test_folder)
print(f"copied {test_files_copied} files to test folder")

print(f"Total groups processed: {len(group_keys)}")
print(f"Groups in train: {len(train_groups)}")
print(f"Groups in test: {len(test_groups)}")

Moved 40870 files to train folder
copied 45280 files to test folder
Total groups processed: 13584
Groups in train: 4528
Groups in test: 4528


In [32]:
# val 폴더로 파일 이동
val_files_copied = copy_files(val, val_groups, val_folder)
print(f"Moved {val_files_copied} files to val folder")

print(f"Total groups processed: {len(val_keys)}")
print(f"Groups in val: {len(val)}")

Moved 5680 files to val folder
Total groups processed: 1704
Groups in val: 568


In [37]:
# Dataset으로 옮겨온 Image와 동일한 이름의 JSON 파일을 label 폴더로 전부 복사
def copy_json_files(base_dir):
    # 처리할 폴더 목록 (Train, Val, Test)
    folders = ['Train', 'Val', 'Test']

    # glob을 사용하여 Training과 Validation의 JSON 파일 목록을 한 번에 가져오기
    json_files = glob.glob('D:\\Falldown\\Origin\\*\\LabelingData\\Image\\**\\*.json', recursive=True)
    
    # json_files를 {파일이름: 경로} 형식의 딕셔너리로 변환
    json_dict = {os.path.splitext(os.path.basename(json_file))[0]: json_file for json_file in json_files}

    for folder in folders:
        images_dir = os.path.join(base_dir, folder, 'Image')
        labels_dir = os.path.join(base_dir, folder, 'Label')

        # label 폴더가 없으면 생성
        if not os.path.exists(labels_dir):
            os.makedirs(labels_dir)

        # Image 폴더의 모든 이미지 파일에 대해
        for image_file in os.listdir(images_dir):
            # 이미지 파일의 이름만 추출 (확장자 제외)
            image_name = os.path.splitext(image_file)[0]

            # 동일한 이름의 JSON 파일이 딕셔너리에 존재하면 labels 폴더로 복사
            if image_name in json_dict:
                json_path = json_dict[image_name]
                destination_path = os.path.join(labels_dir, f"{image_name}.json")
                if not os.path.exists(destination_path):  # 이미 파일이 존재하는지 확인
                    shutil.copy(json_path, destination_path)
                    print(f"Copied {image_name}.json to {labels_dir}")
            else:
                print(f"Warning: JSON file for {image_file} not found")

# 경로 지정
base_directory = 'D:\\Falldown\\Dataset'  # Train, Val, Test 폴더가 있는 기본 디렉토리

copy_json_files(base_directory)

Copied 00001_H_A_SY_C1_I001.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I002.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I003.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I004.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I005.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I006.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I007.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I008.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I009.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C1_I010.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C6_I001.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C6_I002.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C6_I003.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C6_I004.json to D:\Falldown\Dataset\Val\Label
Copied 00001_H_A_SY_C6_I005.json to D:\Falldown\Dataset\Val\Label
Copied 000